# <span style="color:green"><center>Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Transformers- Procesos Internos </center></span>

<center>Behind HuggingFace</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Diseño gráfico y Marketing digital</span>
 

1. Maria del Pilar Montenegro Reyes, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span> 

1. [HuggingFace. Transformers ](https://huggingface.co/transformers/)
1. [HuggingFace. Behind pipeline](https://huggingface.co/course/chapter2/2?fw=tf)
1. [Tutorial Transformer de Google](https://www.tensorflow.org/text/tutorials/transformer)
1. [Transformer-chatbot-tutorial-with-tensorflow-2](https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html) 
1. [Transformer Architecture: The positional encoding](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)
1. [Illustrated Auto-attención](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)
1. [Illustrated Attention](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3#0458)
1. [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf)
1. [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf)
1. [Attention Is All You Need (Vaswani et. al, 2017)](https://arxiv.org/pdf/1706.03762.pdf)
1. [Self-Attention GAN (Zhang et. al, 2018)](https://arxiv.org/pdf/1805.08318.pdf)
1. [Sequence to Sequence Learning with Neural Networks (Sutskever et. al, 2014)](https://arxiv.org/pdf/1409.3215.pdf)
1. [TensorFlow’s seq2seq Tutorial with Attention (Tutorial on seq2seq+attention)](https://github.com/tensorflow/nmt)
1. [Lilian Weng’s Blog on Attention (Great start to attention)](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#a-family-of-attention-mechanisms)
1. [Jay Alammar’s Blog on Seq2Seq with Attention (Great illustrations and worked example on seq2seq+attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
1. [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf)
1. [Adam: A method for stochastic optimization](https://arxiv.org/pdf/1412.6980.pdf)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Etapas del procesamiento natural de texto](#Etapas-del-procesamiento-natural-de-texto)
* [Preprocesamiento con un tokenizador](#Preprocesamiento-con-un-tokenizador)
* [Paso a través del modelo](#Paso-a-través-del-modelo)
* [Postprocesamiento de la salida](#Postprocesamiento-de-la-salida)

## <span style="color:blue">Introducción</span>

Las tareas del procesamiento del lenguaje natural moderno se dividen escencialmente en:

1. Clasificación de textos. Por ejemplo, análisis de sentimiento.
1. Generación automática de textos, basados en una frase inicial.
1. Generación de texto en contexto, llenando espacios vacios enmascarados con máscaras `masked text`.
1. Clasificación de cada una de las palabras en una frase:  Por ejemplo: sustantivo, verbo adjetivo, o por ejemplo `ner`: named entity recognition. ciudad, nombre de persona, localización, organización. 
1. Generación de una respuesta a partir de una pregunta.
1. Traducción de un lenguaje a otro.

## <span style="color:blue">Etapas del procesamiento natural de textos</span>

En esta lección se introducen los procesos que `pipeline` de HuggingFace corre internamente. Básicamente *pipeline* agrupa tres pasos

* Pre-procesamiento.
* Paso a través del modelo.
* Post-procesamiento.

<figure>
<center>
<img src="../Imagenes/full_nlp_pipeline.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Etapas del procesamiento del lenguaje natural</p>
</figcaption>
</figure>

Fuente: [HuggingFace Transformers course](https://huggingface.co/course/chapter2/2?fw=tf)

Para correr el cuaderno original en Tensorflow de HuggingFace en Colab vaya a  [HuggingFace notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/chapter2/section2_tf.ipynb). Versión 
[Pytorch](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/chapter2/section2_pt.ipynb). En esta lección usamos la versión de Tensorflow.

### Clasificación de textos de textos

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my hole life")

[{'label': 'NEGATIVE', 'score': 0.9950999617576599}]

In [3]:
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!"
])

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

## <span style="color:blue">Preprocesamiento con un tokenizador</span>

Como cualquier red neuronal, los transformers no pueden procesar el texto crudo directamente. El primer paso es entonces convertir los textos en números que tengan sentido para la red. Este paso es hecho por un tokenizador. El tokenizador hace lo siguiente:

1. Divide la entrada en palabras, subpalabras y en genral en unidades mínimas textuales como los signos de puntuación. Estas unidades mínimas se denominan tokens.
1. Convierte (*map*) cada token único en un número entero, que será el indice (ID) con el cual se identificará el token a lo largo del proceso.
1. Addiciona símbolos adicionales que será útiles para el modelo.

Para usar un modelo preentrenado, es necesario hacer este proceso exáctamente de la misma manera como se hizo cuando se pre-entrenó al modelo, por lo que es necesario bajar esta información del [Hub del modelo](https://huggingface.co/models).

Esto se hace con la clase AutoTokenizer y su método *from_pretrained*. Se usa el `checkpoint` del modelo, que es el conjunto de pesos del último modelo entrenado.

En HugginFace el checkpoint por defecto para el *pipeline* **sentiment-analysis** es **distilbert-base-uncased-finetuned-sst-2-english**. La tarjeta de presentación del modelo (`card`) se puede consultar [aquí](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english). Veámos:

In [5]:
from transformers import AutoTokenizer

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Ya disponemos del tokenizador adecuado y podemos pasarle nuestras frases para conseguir de vuelta un diccionario que está listo para alimentar al modelo. Lo único que queda por hacer es convertir la lista de ID's a tensores. Los tensores pueden ser Pytorch, Tensorflow y más recientemente Flax. En todo caso es importante tener en cuenta que los transformers en Hugginface solamente aceptan tensores. 

In [6]:
raw_inputs = [
    'This course is fantastic.',
    'I hate this so much!'
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='tf') # return ='pt' para tensores pytorch
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[  101,  2023,  2607,  2003, 10392,  1012,   102,     0],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


### Notas

1. Se han agregado los token de inicio de sentencia(101) y fin de sentencia(102). La primera sentencia tiene un padding (relleno) al cual se asigna el código 0. 
1. La máscara de atención marca los tokens de relleno con cero. Esto se usa en la red transformer para enmascarar los token  de relleno de tal manera que el softmax que codifica finalmente la autoatención no le de peso a esos token.

## <span style="color:blue">Paso a través del modelo</span>

Podemos bajar el modelo preentrenado mediante el uso de la clase *TFAutoModel* que también tiene el método *from-trained*.

In [8]:
from transformers import TFAutoModel # AutoModel en Pythorch

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model = TFAutoModel.from_pretrained(checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['pre_classifier', 'classifier', 'dropout_19']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
model.summary()

Model: "tf_distil_bert_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0
_________________________________________________________________


### Vector de salida 

Es usualmente un tensor de tamaño grande y con tres dimensiones:

* Tamaño del lote (**Batch size**): tamaño de secuencias a procesar cada vez.
* Longitud de secuencia (**sequence length**) Tamaño de cada secuencia.
* Dimensión del modelo (**Hidden size**): tamaño del embedding. Usualmente 768 para modelos pequeños y hasta 3072 para modelos grandes)

In [10]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 8, 768)


In [7]:
outputs

TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(2, 8, 768), dtype=float32, numpy=
array([[[ 0.71443266,  0.16590782,  0.22164305, ...,  0.4517184 ,
          0.8582398 , -0.4670435 ],
        [ 0.833101  ,  0.1852059 ,  0.14624734, ...,  0.42707193,
          0.8372486 , -0.39773327],
        [ 0.9286826 ,  0.1862842 ,  0.2808665 , ...,  0.38525626,
          0.7030744 , -0.4284996 ],
        ...,
        [ 0.8273146 ,  0.05676208,  0.45793867, ...,  0.43878704,
          0.83444375, -0.67980033],
        [ 1.0110863 ,  0.14425977,  0.80684364, ...,  0.4608204 ,
          0.61149776, -0.76684564],
        [ 0.6806943 ,  0.1321691 ,  0.23668604, ...,  0.40009487,
          0.85364753, -0.45703   ]],

       [[-0.29370627,  0.7282562 , -0.14972652, ..., -0.11868126,
         -1.0226722 , -0.04215673],
        [-0.22063597,  0.9383844 , -0.09512459, ..., -0.36431718,
         -0.66052145,  0.24069718],
        [-0.15360779,  0.89875007, -0.07276374, ..., -0.21891779,
         -0.85

El vector de salida *outputs* contiene la salida de la red transformer. Es decir la salida de la capa codificadora en este caso de clasificación de sentencias en positivas o negativas.

### Cabezas del modelo (model heads)

En los modelos de HuggingFace la  cabeza corresponde al conjunto final de capas lineales que dan sentido a la tarea a realizar. La imagen ilustra el concepto. Todo el mecanismo de auto-atención está incorporado en la red transformer. En otras palabras, la cabeza es la parte que se agrega al modelo pre-entrenado, luego de quitarle la capa final, para luergo hacer el ajuste fino (fine tunning).

<figure>
<center>
<img src="../Imagenes/transformer_and_head.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo transformer en huggingface</p>
</figcaption>
</figure>

Fuente: [HuggingFace Transformers course](https://huggingface.co/course/chapter2/2?fw=tf)

Hay diferentes arquitecturas disponibles en los transformers de HuggingFace, cada diseñada para atacar una tarea específica. Algunas de estas tareas son:
   
* Model: para recupera los estados ocultos al final de la capa transformer(retrieve the hidden states)
* ForCausalLM
* ForMaskedLM
* ForMultipleChoice
* ForQuestionAnswering
* ForSequenceClassification
* ForTokenClassification
* Otras

Por ejemplo para la tarea de clasificación no usaremos Model (TFAutoModel) que mostramos arriba, sino que usaremos *TFAutoModelForSequenceClassification*.

In [12]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(inputs)

print(outputs.logits.shape)


Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_57']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(2, 2)


## <span style="color:blue">Pos-procesamiento de la salida</span>

La salida en nuestro ejmplo son logits:

In [15]:
print(outputs.logits)

tf.Tensor(
[[-4.358263   4.692012 ]
 [ 4.169232  -3.3464477]], shape=(2, 2), dtype=float32)


El modelo predice [-4.358263,   4.692012 ] para la primera sentencia y [ 4.169232,  -3.3464477].  Estos valores no son probabilidades sino logits. Para convertirlas a probabilidades se usa la función *softmax*.

In [17]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits)
print(predictions)

tf.Tensor(
[[1.17345015e-04 9.99882698e-01]
 [9.99455869e-01 5.44183713e-04]], shape=(2, 2), dtype=float32)


Para obtener las etiquetas (labels) asociadas a cada posición se puede utilizar *model.config.id2label*.

In [16]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Por lo que la primera sentencia es calificada como positiva y la segunda como negativa.

</span>